<a href="https://colab.research.google.com/github/myrah/AAI2025/blob/main/LangGraph/QnAagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade --quiet  langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from openai import OpenAI
from google.colab import userdata

# Set your OpenAI API key (from https://platform.openai.com/api-keys)
api_key=userdata.get("OPENAI_EDU_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key  # optional but recommended

# --- Setup the LLM ---
model = ChatOpenAI(
    model="gpt-4o",        # or "gpt-4o-mini", "gpt-4-turbo", etc.
    temperature=0.7        # optional, controls randomness
)

# --- Setup the Embedding model ---
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

In [ ]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the pricxe of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
#print(get_laptop_price("alpha"))
#print(get_laptop_price("testing"))

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


In [ ]:
!pip install --upgrade --quiet  langgraph
!pip install --upgrade --quiet  pypdf
!pip install --upgrade --quiet  langchain-chroma

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
#print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)

In [ ]:
# Example of how to invoke the agent
# Replace the question with your query about laptops
response = product_QnA_agent.invoke(
    {"messages": [HumanMessage(content="What is the price of AlphaBook Pro?")]},
    config={"configurable": {"thread_id": "some-thread-id"}} # Replace with a unique thread_id for each conversation
)

# Print the response from the agent
print(response)

{'messages': [HumanMessage(content='What is the price of AlphaBook Pro?', additional_kwargs={}, response_metadata={}, id='c5ad54a3-ca64-49bc-9b52-a21adeb6abc9'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BvBwrswo07ZvYvQeRHVUotne', 'function': {'arguments': '{"laptop_name":"AlphaBook Pro"}', 'name': 'get_laptop_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 159, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_1827dd0c55', 'id': 'chatcmpl-CO7KwSdL5W87g40f59XPyGpu66MvM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ba246df9-4674-404f-a697-944b4526f71d-0', tool_calls=[{'name': 'get_laptop_price', 'args': {'lap

In [ ]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_cE1LTtb9UJDzIVfxig1n5jbU)
 Call ID: call_cE1LTtb9UJDzIVfxig1n5jbU
  Args:
    query: GammaAir
  get_laptop_price (call_jOucP9q0JBUT14BDJckvNDta)
 Call ID: call_jOucP9q0JBUT14BDJckvNDta
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

The GammaAir X is a laptop that combines the following features:

- **Processor**: AMD Ryzen 7
- **Memory**: 32GB of DDR4 RAM
- **Storage**: 512GB NVMe SSD
- **Design**: Thin and light form factor, perfect for users who need high performance in a portable design

The price for the GammaAir X is $1,399.


In [ ]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I can help you with that. Are there specific features or specifications you're looking for in a laptop? For example, CPU type, memory size, storage capacity, or any particular design features? Let me know your preferences so I can provide more tailored recommendations.
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here is a list of available laptops:

1. **AlphaBook Pro**
   - 12th Gen Intel i7 processor, 16GB DDR4 RAM, 1TB SSD.

2. **GammaAir X**
   - AMD Ryzen 7 processor, 32GB DDR4 RAM, 512GB NVMe SSD.

3. **SpectraBook S**
   - Intel Core i9 processor, 64GB RAM, 2TB SSD.

4. **OmegaPro G17**
   - Ryzen 9 5900HX CPU, 32GB RAM, 1TB SSD.

5. **NanoEdge Flex**

Let me know if you need more details about any of these laptops or if you want to know the

In [ ]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")


USER 1: The SpectraBook S is designed for power users and comes with the following features:

- **Processor:** Intel Core i9
- **Memory:** 64GB RAM
- **Storage:** 2TB SSD

This workstation-class laptop is ideal for intensive tasks such as video editing and 3D rendering.

USER 2: The GammaAir X is a laptop that features:

- **Processor**: AMD Ryzen 7
- **Memory**: 32GB of DDR4 RAM
- **Storage**: 512GB NVMe SSD
- **Design**: Thin and light form factor, making it highly portable yet powerful.

This laptop is ideal for users who require high performance in a portable design.

USER 1: The price of the SpectraBook S is $2,499.

USER 2: The price of the GammaAir X laptop is $1399.
